# Modificndo a los datos y definiendo restricciones
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### 1. Introducción

Durante esta actividad ocuparemos comandos básicos de SQL para explorar datos en una tabla guardando información sobre películas. Adicionalmente, aprenderemos cómo modificar o insertar una o varias tuplas. 

### 1.1 Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.

### 1.2 Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

con la llave primaria `pid`.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

### Creando nuestra base de datos

Para crear nuestra tabla, y insertar tuplas con cuales trabajaremos, hay que ejecutar el siguiente bloque de comandos SQL.

In [ ]:
%%sql

CREATE TABLE Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

INSERT INTO Peliculas VALUES(1,'Avengers',2019,'SciFi',8.4,'Brothers Russo');
INSERT INTO Peliculas VALUES(2,'Captain America',2016,'SciFi',8.7,'Brothers Russo');
INSERT INTO Peliculas VALUES(3,'Iron Man',2008,'SciFi',9.1,'John Favreu');
INSERT INTO Peliculas VALUES(4,'Batman',2008,'Thriller',8.3,'Christoper Nolan');
INSERT INTO Peliculas VALUES(5,'Batman',2012,'Thriller',7.9,'Christoper Nolan');
INSERT INTO Peliculas VALUES(6,'Interstellar',2014,'Drama',8.4,'Christoper Nolan');
INSERT INTO Peliculas VALUES(7,'Sherlok Holmes',2009,'Mystery',7.8,'Guy Ritchie');

SELECT * FROM Peliculas;

**Cuidado**: cada vez que anteponemos `%sql` es porque esa línea corresponde a un comando a SQL que va a la base de datos en la que estamos trabajando. Si queremos tener todo un bloque con instrucciones SQL tenemos que usar `%%sql` (habrán ejemplos de esto a lo largo del *notebook*).

### Explorando datos con SQL

Ya sabemos cómo encontrar varia información en nuestra tabla `Películas`. Por ejemplo, si nos interesan solo las películas de la categoría `SciFi`, podemos escribir:

In [ ]:
%%sql

SELECT * 
FROM Peliculas
WHERE Peliculas.pcategoria = 'SciFi';

Una funcionalidad útil es ocupar las palabras clave `BETWEEN`, o `IN` en un filtro. Por ejemplo, si nos interesan solo las películas estrenadas entre el año 2015 y 2020, podemos escribir:

In [ ]:
%%sql

SELECT *
FROM Peliculas
WHERE Peliculas.paño BETWEEN 2015 AND 2020;

Para selecionar las películas estrenadas en los años 2008, 2012, o 2014, podemos ocupa `IN`:

In [ ]:
%%sql

SELECT *
FROM Peliculas
WHERE Peliculas.paño IN (2008,2012,2014);

### Alias

Para no tener que escribir el nombre entero de la tabla en cada condición, siempre podemos ocupar aliases para tablas, especificados con el comando `AS`. Por ejemplo, si queremos los nombres de las películas estrenadas en los años 2008, 2012, o 2014, podemos escribir:

In [ ]:
%%sql 

SELECT P.pnombre
FROM Peliculas AS P
WHERE P.paño IN (2008,2012,2014);

Si no nos gusta el nombre de un atributo en la relación, al devolverlos, `AS` también permite cambiar su nombre. Por ejemplo, si en vez de `pnombre` queremos desplegar `name`, podemos escribir:

In [ ]:
%%sql 

SELECT P.pnombre AS name
FROM Peliculas AS P
WHERE P.paño IN (2008,2012,2014);

Es importante observar que el comando `AS` no cambia el nombre de la tabla base, ni del atributo en la misma tabla. `AS` solo define este nombre en el contexto de la consulta actual.

### Actualizando a los datos

Muchas veces es necesario cambiar valor de un atributo en una tupla. Por ejemplo, si la calificación de la película `Interstellar` cambia de `8.4` a `8.9`, necesitamos actualizar el valor de atributo `pcalificaion` en la tupla correspondiente a la película `Interstellar`.

En general, la forma para hacer esto en SQL es:
```SQL
UPDATE Tabla
SET Nuevos Valores
WHERE Condición sobre Tabla
```

Intuitivamente, el comando dice actualizar la tabla especificada ene el `UPDATE`, especificando los valores nuevos en el `SET`, para las tuplas de la tabla selecionadas por la condición en `WHERE`.

En nuestro ejemplo, podemos escribir:

In [ ]:
%%sql

UPDATE Peliculas
SET pcalificacion = 8.9
WHERE pnombre = 'Interstellar';

Revisando nuestra tabla, podemos observar el cambio:

In [ ]:
%%sql
SELECT * FROM Peliculas;

Es importante especificar bien la condicion en el `WHERE` para no cambiar valores no deseados. Por ejemplo, si nos olvidamos la clúsula `WHERE`, actualizaremos la tabla entera:

In [ ]:
%%sql

UPDATE Peliculas
SET pcalificacion = 8.9;

SELECT * FROM Peliculas;

Para eliminar una tupla, ovarias tuplas, la sintáxis de SQL es:

```SQL
DELETE From Tabla
WHERE Condición sobre Tabla
```

Este comando simplemente eliminaría todas las tuplas que satisfacen la condición especificada en el `WHERE`.

Por ejemplo, para eliminar todas las películas de categoría `Mystery`, escribiremos:

In [ ]:
%%sql

DELETE FROM Peliculas
WHERE Peliculas.pcategoria = 'Mystery';

Esto nos deja con:

In [ ]:
%%sql
SELECT * FROM Peliculas;

De nuevo, es importante especificar bien la condición en el `WHERE` para no borrar la tabla entera:

In [ ]:
%%sql

DELETE FROM Peliculas
WHERE Peliculas.pcategoria LIKE '%';

SELECT * FROM Peliculas;

Es importante observar que al borrar todas las tuplas, la tabla todavía existe en el sistema, y podemos insertar tuplas nuevas, o counsultar a la tabla:

In [ ]:
%%sql

INSERT INTO Peliculas VALUES(7,'Sherlok Holmes',2009,'Mystery',7.8,'Guy Ritchie');

SELECT * FROM Peliculas;

Para eliminar la tabla, debemos ejecutar:

In [ ]:
%%sql
DROP TABLE Peliculas;

Ahora la tabla no existe, y no la podemos llenar con tuplas, ni consultar:

In [ ]:
%%sql

INSERT INTO Peliculas VALUES(7,'Sherlok Holmes',2009,'Mystery',7.8,'Guy Ritchie');

In [ ]:
%%sql

SELECT * FROM Peliculas;

### Resumen
En este tutorial vimos cómo:

- Recuperar la información desde una base de datos con SQL
- Poner nombres temporales a tablas y atributos con `AS`
- Actualizar tuplas de una tabla con `UPDATE`
- Eliminar tuplas y tablas.

En las siguienetes clases exploraremos cómo recuperar la información desde más de una tabla al mismo tiempo.